In [33]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
from numba import jit

In [3]:
import dask.dataframe as dd
from dask.distributed import Client

client = Client()
client

/Users/Copeland/opt/anaconda3/lib/python3.9/site-packages/distributed/node.py:179: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 57176 instead
  warnings.warn(
2024-03-20 15:56:27,199 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/1j/227gtx89455_vcjz4mqv09ch0000gn/T/dask-worker-space/worker-pzxn60rr', purging
2024-03-20 15:56:27,205 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/1j/227gtx89455_vcjz4mqv09ch0000gn/T/dask-worker-space/worker-vwax0j59', purging
2024-03-20 15:56:27,207 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/1j/227gtx89455_vcjz4mqv09ch0000gn/T/dask-worker-space/worker-wv104onw', purging
2024-03-20 15:56:27,208 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/1j/227gtx89455_vcjz4mqv09ch0000gn/T/dask-worker-space/worker-0l7x54_5', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:57176/status,
Dashboard: http://127.0.0.1:57176/status,Workers: 4
Total threads: 4,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:57177,Workers: 4
Dashboard: http://127.0.0.1:57176/status,Total threads: 4
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:57194,Total threads: 1
Dashboard: http://127.0.0.1:57196/status,Memory: 8.00 GiB
Nanny: tcp://127.0.0.1:57181,


### Read in the and data drop irrelevant features

In [4]:
dft = pd.read_csv('us_congestion_2016_2022.csv')

/var/folders/1j/227gtx89455_vcjz4mqv09ch0000gn/T/ipykernel_61529/418031929.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  dft = pd.read_csv('us_congestion_2016_2022.csv')


In [5]:
dfw = pd.read_csv('WeatherEvents_Jan2016-Dec2022.csv')

In [6]:
# dft.to_parquet('us_congestion_2016_2022')
# dfw.to_parquet('WeatherEvents_Jan2016-Dec2022')
# df = dd.read_parquet('path/to/my-results/')

In [7]:
dft = dft.drop(['ID', 'Description'], axis=1)
dfw = dfw.drop(['EventId', 'AirportCode'], axis=1)

### 'Severity' column is numerical but still categorical, change it to a string type

In [8]:
dft['Severity'] = dft['Severity'].astype('string')

In [9]:
dft.head()

,Severity,Start_Lat,Start_Lng,StartTime,EndTime,Distance(mi),DelayFromTypicalTraffic(mins),DelayFromFreeFlowSpeed(mins),Congestion_Speed,Street,...,Temperature(F),WindChill(F),Humidity(%),Pressure(in),Visibility(mi),WindDir,WindSpeed(mph),Precipitation(in),Weather_Event,Weather_Conditions
0,0,42.410881,-71.147995,2020-11-10T17:54:00.000-05:00,2020-11-10T19:29:27.000-05:00,3.99,0.0,4.0,Moderate,Wyman Ter,...,64.0,64.0,63.0,29.93,10.0,SSW,7.0,0.0,NaN,Fair
1,0,42.512131,-71.027992,2020-11-10T17:53:00.000-05:00,2020-11-10T18:33:43.000-05:00,0.75,1.0,1.0,Moderate,Walnut St,...,66.0,66.0,54.0,29.95,10.0,S,3.0,0.0,NaN,Fair
2,1,42.272388,-71.068001,2020-11-10T17:49:00.000-05:00,2020-11-10T18:39:39.000-05:00,4.28,0.0,7.0,Slow,Dorchester Ave,...,65.0,65.0,70.0,30.05,10.0,S,7.0,0.0,NaN,Fair
3,0,42.356911,-71.066254,2020-11-10T17:58:00.000-05:00,2020-11-10T18:39:39.000-05:00,0.38,0.0,2.0,Slow,Beacon St,...,63.0,63.0,70.0,30.07,10.0,SSW,10.0,0.0,NaN,Mostly Cloudy
4,1,42.377628,-71.120079,2020-11-10T17:46:00.000-05:00,2020-11-10T18:39:39.000-05:00,3.55,0.0,5.0,Moderate,Massachusetts Ave,...,63.0,63.0,70.0,30.07,10.0,SSW,10.0,0.0,NaN,Mostly Cloudy


In [10]:
dfw.head()

,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,LocationLat,LocationLng,City,County,State,ZipCode
0,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,0.00,US/Mountain,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
1,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,0.00,US/Mountain,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
2,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,0.03,US/Mountain,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
3,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,0.00,US/Mountain,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
4,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,0.00,US/Mountain,38.0972,-106.1689,Saguache,Saguache,CO,81149.0


### Take a look at the different data types

In [11]:
dft_num = dft.select_dtypes(include='number')
dfw_num = dfw.select_dtypes(include='number')

In [33]:
def get_discriptions(df):
    for column in df:
        print(df[column].describe())
        print()
        
    return

In [34]:
get_discriptions(dft_num)

count    3.330420e+07
mean     3.717374e+01
std      5.171325e+00
min      2.454662e+01
25%      3.377818e+01
50%      3.846994e+01
75%      4.084970e+01
max      4.900249e+01
Name: Start_Lat, dtype: float64

count    3.330420e+07
mean    -9.235009e+01
std      1.711851e+01
min     -1.245524e+02
25%     -1.063377e+02
50%     -8.686362e+01
75%     -7.742982e+01
max     -6.701227e+01
Name: Start_Lng, dtype: float64

count    3.330420e+07
mean     2.989116e+00
std      2.880462e+00
min      0.000000e+00
25%      1.260000e+00
50%      2.320000e+00
75%      3.780000e+00
max      2.121200e+02
Name: Distance(mi), dtype: float64

count    3.330420e+07
mean     2.816910e+00
std      4.070466e+00
min      0.000000e+00
25%      5.500000e-01
50%      1.920000e+00
75%      3.950000e+00
max      3.660000e+02
Name: DelayFromTypicalTraffic(mins), dtype: float64

count    3.330420e+07
mean     4.020801e+00
std      4.585300e+00
min      0.000000e+00
25%      1.800000e+00
50%      3.000000e+00
75%      

In [35]:
get_discriptions(dfw_num)

count    8.627181e+06
mean     9.287441e-02
std      8.870326e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      5.000000e-02
max      1.104130e+03
Name: Precipitation(in), dtype: float64

count    8.627181e+06
mean     3.878602e+01
std      5.467708e+00
min      2.455710e+01
25%      3.459950e+01
50%      3.934510e+01
75%      4.306220e+01
max      4.894020e+01
Name: LocationLat, dtype: float64

count    8.627181e+06
mean    -9.191172e+01
std      1.350340e+01
min     -1.245550e+02
25%     -9.782360e+01
50%     -8.977340e+01
75%     -8.194720e+01
max     -6.779280e+01
Name: LocationLng, dtype: float64

count    8.557982e+06
mean     5.241150e+04
std      2.573249e+04
min      1.022000e+03
25%      3.121600e+04
50%      5.391300e+04
75%      7.350300e+04
max      9.936200e+04
Name: ZipCode, dtype: float64



In [12]:
dft_cat = dft.select_dtypes(exclude='number')
dfw_cat = dfw.select_dtypes(exclude='number')

In [37]:
def get_nunique(df):
    for column in df.columns:
        print(f'Unique values in column "{column}": {df[column].nunique()}')
        
    return

In [38]:
get_nunique(dft_cat)

Unique values in column "Severity": 5
Unique values in column "StartTime": 2913336
Unique values in column "EndTime": 13790299
Unique values in column "Congestion_Speed": 3
Unique values in column "Street": 234753
Unique values in column "City": 16776
Unique values in column "County": 1973
Unique values in column "State": 49
Unique values in column "Country": 1
Unique values in column "ZipCode": 1293808
Unique values in column "LocalTimeZone": 4
Unique values in column "WeatherStation_AirportCode": 2058
Unique values in column "WeatherTimeStamp": 1466130
Unique values in column "WindDir": 24
Unique values in column "Weather_Event": 26
Unique values in column "Weather_Conditions": 173


In [39]:
get_nunique(dfw_cat)

Unique values in column "Type": 7
Unique values in column "Severity": 6
Unique values in column "StartTime(UTC)": 2285322
Unique values in column "EndTime(UTC)": 2237800
Unique values in column "TimeZone": 4
Unique values in column "City": 1716
Unique values in column "County": 1100
Unique values in column "State": 48


### Lets check for null values

In [40]:
def get_missing(df):
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)

    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    
    return missing_data

In [41]:
missing_dft = get_missing(dft)
missing_dft

,Total,Percent
Weather_Event,31165872,9.357941e-01
Precipitation(in),13753401,4.129630e-01
WindChill(F),12655165,3.799871e-01
WindSpeed(mph),3159455,9.486657e-02
Visibility(mi),973275,2.922379e-02
Weather_Conditions,949187,2.850052e-02
WindDir,789678,2.371106e-02
Humidity(%),766191,2.300584e-02
Temperature(F),729816,2.191363e-02
Pressure(in),682651,2.049745e-02


In [42]:
missing_dfw = get_missing(dfw)
missing_dfw

,Total,Percent
ZipCode,69199,0.008021
City,16912,0.001960
Type,0,0.000000
Severity,0,0.000000
StartTime(UTC),0,0.000000
EndTime(UTC),0,0.000000
Precipitation(in),0,0.000000
TimeZone,0,0.000000
LocationLat,0,0.000000
LocationLng,0,0.000000


How many data points can we save by dropping different columns from the traffic data?

In [13]:
df_drop1 = dft.drop('Weather_Event', axis=1)
df_drop2 = df_drop1.drop('Precipitation(in)', axis=1)
df_drop3 = df_drop2.drop('WindChill(F)', axis=1)
df_drop4 = df_drop3.drop('WindSpeed(mph)', axis=1)

In [14]:
df_drop1 = df_drop1.dropna()
df_drop2 = df_drop2.dropna()
df_drop3 = df_drop3.dropna()
df_drop4 = df_drop4.dropna()

In [16]:
def data_saved(no_null_df, original_df):
    print(f'Preserves {len(no_null_df.axes[0]) / len(original_df.axes[0]) * 100}% of data points or {len(no_null_df.axes[0])} rows')
    
    return

data_saved(df_drop1, dft)
data_saved(df_drop2, dft)
data_saved(df_drop3, dft)
data_saved(df_drop4, dft)

Preserves 53.87703214240342% of data points or 17943314 rows
Preserves 61.212896908284755% of data points or 20386465 rows
Preserves 89.10155142899548% of data points or 29674558 rows
Preserves 95.65413658499938% of data points or 31856844 rows


### Versus dropping all NaN rows
One thing I notice after dropping missing rows is that Weather_Conditions could be a good standin for Weather_Event

In [17]:
dft_drop_rows = dft.dropna()

data_saved(dft_drop_rows, dft)

Preserves 2.736699957864172% of data points or 911436 rows


For the weather df, we can just drop the rows and see how many we have left

In [18]:
dfw_drop_rows = dfw.dropna()

data_saved(dfw_drop_rows, dfw)

Preserves 99.19789558141878% of data points or 8557982 rows


In [19]:
print('Number of NaN values in dft:', df_drop2.isnull().sum().sum())
print('Number of NaN values in dfw:', dfw_drop_rows.isnull().sum().sum())

Number of NaN values in dft: 0
Number of NaN values in dfw: 0


In [20]:
dft = df_drop2.copy()
dft.dtypes

Severity                          string
Start_Lat                        float64
Start_Lng                        float64
StartTime                         object
EndTime                           object
Distance(mi)                     float64
DelayFromTypicalTraffic(mins)    float64
DelayFromFreeFlowSpeed(mins)     float64
Congestion_Speed                  object
Street                            object
City                              object
County                            object
State                             object
Country                           object
ZipCode                           object
LocalTimeZone                     object
WeatherStation_AirportCode        object
WeatherTimeStamp                  object
Temperature(F)                   float64
WindChill(F)                     float64
Humidity(%)                      float64
Pressure(in)                     float64
Visibility(mi)                   float64
WindDir                           object
WindSpeed(mph)  

In [21]:
dfw = dfw_drop_rows.copy()
dfw.dtypes

Type                  object
Severity              object
StartTime(UTC)        object
EndTime(UTC)          object
Precipitation(in)    float64
TimeZone              object
LocationLat          float64
LocationLng          float64
City                  object
County                object
State                 object
ZipCode              float64
dtype: object

### 'StartTime', 'EndTime', and 'WeatherTimeStamp' should be converted to DateTime
This could come in handy when we do our time series analysis

In [22]:
# ddft['StartTime'] = dd.to_datetime(ddft['StartTime'])
dft['StartTime'] = pd.to_datetime(dft['StartTime'], utc=True)

In [23]:
# ddft['EndTime'] = dd.to_datetime(ddft['EndTime'])
dft['EndTime'] = pd.to_datetime(dft['EndTime'], utc=True)

In [24]:
# ddft['WeatherTimeStamp'] = dd.to_datetime(ddft['WeatherTimeStamp'])
dft['WeatherTimeStamp'] = pd.to_datetime(dft['WeatherTimeStamp'], utc=True)

### Try the join with Dask and Pandas

In [25]:
# dft.to_csv('traffic.csv')
# dfw.to_csv('weather.csv')

In [26]:
# ddft = dd.read_csv('traffic.csv')
# ddfw = dd.read_csv('weather.csv')

In [36]:
dft.to_parquet('traffic.parquet')
dfw.to_parquet('weather.parquet')

In [37]:
ddft = dd.read_parquet('traffic.parquet')
ddfw = dd.read_parquet('weather.parquet')

In [40]:
ddft.columns

Index(['Severity', 'Start_Lat', 'Start_Lng', 'StartTime', 'EndTime',
       'Distance(mi)', 'DelayFromTypicalTraffic(mins)',
       'DelayFromFreeFlowSpeed(mins)', 'Congestion_Speed', 'Street', 'City',
       'County', 'State', 'Country', 'ZipCode', 'LocalTimeZone',
       'WeatherStation_AirportCode', 'WeatherTimeStamp', 'Temperature(F)',
       'WindChill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)',
       'WindDir', 'WindSpeed(mph)', 'Weather_Conditions'],
      dtype='object')

In [41]:
ddfw.columns

Index(['Type', 'Severity', 'StartTime(UTC)', 'EndTime(UTC)',
       'Precipitation(in)', 'TimeZone', 'LocationLat', 'LocationLng', 'City',
       'County', 'State', 'ZipCode'],
      dtype='object')

In [42]:
ddf = dd.merge(ddft, ddfw, how="inner", on=["State", "City"])

In [ ]:
# ddf = ddf.compute()

In [ ]:
# ddf.to_csv('joined_data.csv')

In [46]:
ddf.columns

Index(['Severity_x', 'Start_Lat', 'Start_Lng', 'StartTime', 'EndTime',
       'Distance(mi)', 'DelayFromTypicalTraffic(mins)',
       'DelayFromFreeFlowSpeed(mins)', 'Congestion_Speed', 'Street', 'City',
       'County_x', 'State', 'Country', 'ZipCode_x', 'LocalTimeZone',
       'WeatherStation_AirportCode', 'WeatherTimeStamp', 'Temperature(F)',
       'WindChill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)',
       'WindDir', 'WindSpeed(mph)', 'Weather_Conditions', 'Type', 'Severity_y',
       'StartTime(UTC)', 'EndTime(UTC)', 'Precipitation(in)', 'TimeZone',
       'LocationLat', 'LocationLng', 'County_y', 'ZipCode_y'],
      dtype='object')

At this point i think I've done everything i can locally, ther est will have to be done in another way. If we wan to reduce the size of the data, maybe we can move back to working on it locally. 

# stop here

### We might want access to the various date and time components separately
Im running the rest of these tests of the dataframe df_drop_we, where Weather_Events was dropped before null rows were dropped. The same transformations can be done to any final, clean dataset we decide to use.

In [ ]:
def extract_datetime_info(datetime_obj):
    
    return datetime_obj.year, datetime_obj.month, datetime_obj.day, datetime_obj.time()

df_drop_we[['Year', 'Month', 'Day', 'StartTOD']] = df_drop_we['StartTime'].apply(
    lambda x: pd.Series(extract_datetime_info(x))
)

In [ ]:
df_drop_we['EndTOD'] = df_drop_we['EndTime'].apply(
    lambda x: pd.Series((pd.to_datetime(x).time()))
)

In [ ]:
df_drop_we['WeatherTime'] = df_drop_we['WeatherTimeStamp'].apply(
    lambda x: pd.Series((pd.to_datetime(x).time()))
)

In [ ]:
df_drop_we